In [42]:
%%capture
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter
from glob import glob
import csv
import mat73
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [43]:
from scipy import stats
import entropy as ent
import statsmodels.api as sm
import scipy.io
import numpy as np
import librosa
from scipy.signal import find_peaks
from scipy.stats import differential_entropy

def mean(x):
    return np.mean(x)

def std(x):
    return np.std(x)

def mad(x):
    return stats.median_abs_deviation(x)

def mixim(x):
    return np.max(x)

def minim(x):
    return np.min(x)

def SMA(x):
    return sum(np.sqrt(x**2+x**2))/len(x)

def energy(x):
    return np.sum(np.square(x))/len(x)

def iqr(x):
    return np.subtract(*np.percentile(x, [75, 25]))

def spectral_entropy(x,fs=1000):
    return ent.spectral_entropy(x, sf=fs, method='welch', normalize=True)

def arCoeff(x):
    rho, sigma2 = sm.regression.linear_model.burg(x, order=1)
    return rho[0]

def argmaxim(x):
    return np.argmax(x)

def skewness(x):
    return stats.skew(x)

def kurtosis(x):
    return stats.kurtosis(x)

def bandenergy(x,Fs=1000):
    (fq, Se) = scipy.signal.periodogram(x, Fs, scaling='density')
    return sum(Se)

def ptp(x):
    return np.ptp(x)

def var(x):
    return np.var(x)

def argminim(x):
    return np.argmin(x)

def rms(x):
    return np.sqrt(np.mean(x**2))

def abs_diff_signal(x):
    return np.sum(np.abs(np.diff(x)))

def zcrs(x):
    return librosa.feature.zero_crossing_rate(x)[0][0]

def diff_entropy(x):
    return differential_entropy(x)

def mobility(x):
    return ent.hjorth_params(x)[0]

def complexity(x):
    return ent.hjorth_params(x)[1]

def peak_ampl(x):
    peaks, _ = find_peaks(x, height=0)
    return (max(peaks))

def sample_entropy(x):
    return ent.sample_entropy(x)

def approx_entropy(x):
    return ent.app_entropy(x)

def sing_value_decomp_entropy(x):
    return ent.svd_entropy(x, normalize=True)

def permuation_entropy(x):
    return ent.perm_entropy(x, normalize=True)

def all_features(x,fs=1000):
    k = [mean(x),std(x),mad(x),mixim(x),minim(x),SMA(x),energy(x),iqr(x),spectral_entropy(x,fs),arCoeff(x),skewness(x),kurtosis(x),bandenergy(x,fs),ptp(x),var(x),rms(x),
         abs_diff_signal(x),zcrs(x),diff_entropy(x),mobility(x),complexity(x),peak_ampl(x),sample_entropy(x),approx_entropy(x),sing_value_decomp_entropy(x),permuation_entropy(x)]
    return k

In [44]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    return butter(order, [lowcut, highcut], fs=fs, btype='band')

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y
def filter_(xx,fs):
    xx_fastripples = butter_bandpass_filter(xx,250,499,fs,order=6)
    xx_ripples = butter_bandpass_filter(xx,80,250,fs,order=6)
    return xx_fastripples,xx_ripples

In [45]:
def dictionary(file_name):
    with open('Patients/mat/'+file_name+'.txt', 'r') as f:
        txt_file = f.read()
    txt_file = txt_file.split('\n')
    index_array = np.arange(1,len(txt_file))
    dictionary = dict(zip(index_array,txt_file))
    return dictionary

In [46]:
def R_FR_features(mat,xx_fastripples,xx_ripples,t_hold,fs,patient_id,k):
    R_features=[]
    FR_features=[]
    xx = mat['xx']
    #len(mat['Ripples'])
#     for i in range(4,5):
    for i in range(len(mat['Ripples'])):
#         print(mat['Ripples'][i][4])
        if mat['Ripples'][i][4] >= t_hold:
            channel_no = mat['Ripples'][i][0]
            index_xx = int(channel_no - 1)
            if mat['Ripples'][i][2] > 1100 and mat['Ripples'][i][2]<(int(xx.shape[1])-1100):
                start = int(mat['Ripples'][i][2]-1000)
                stop = int(mat['Ripples'][i][2]+1000)
            else:
                continue
            R_seg = xx_ripples[index_xx][start:stop]
#             print(len(R_seg))
#             time = np.linspace(0,10,len(R_seg))
#             plt.plot(time,R_seg)
#             plt.show()
#             print('done')
            features = all_features(R_seg,fs)
            features = np.hstack((patient_id[k],mat['Ripples'][i][0],features,mat['Ripples'][i][1]))
            R_features.append(features)
    
    for i in range(3,len(mat['FastRipples'])):
        if mat['FastRipples'][i][4] >= t_hold:
            channel_no = mat['FastRipples'][i][0]
            index_xx = int(channel_no - 1)
            if mat['FastRipples'][i][2] > 1100 and mat['FastRipples'][i][2]<(int(xx.shape[1])-1100):
                start = int(mat['FastRipples'][i][2]-1000)
                stop = int(mat['FastRipples'][i][2]+1000)
            else:
                continue
            FR_seg = xx_fastripples[index_xx][start:stop]
            features = all_features(FR_seg,fs)
            features = list(np.hstack((patient_id[k],mat['FastRipples'][i][0],features,mat['FastRipples'][i][1])))
            FR_features.append(features)
    R_features = np.array(R_features) 
    FR_features = np.array(FR_features)
    return R_features,FR_features

In [47]:
def write_csv(file_names,i,R_features,FR_features):
    file = open(file_names[i]+'2000_R_features.csv', 'w+', newline ='')
    with file:   
        write = csv.writer(file)
        write.writerows(R_features)
    file = open(file_names[i]+'2000_FR_features.csv', 'w+', newline ='')
    with file:   
        write = csv.writer(file)
        write.writerows(FR_features)
    

In [48]:
def label(R_features,index_no,k):
    types = index_no[k].split(',')
    if len(types)>1:
        set_array = np.array([])
        for i in range(len(types)):
            types_2 = types[i]
            values = types[i].split('-')
            set_values = np.arange(int(values[0]),int(values[1])+1)
            set_array= np.hstack((set_array,set_values))
    else:
        values = index_no[k].split('-')
        set_array = np.arange(int(values[0]),int(values[1])+1)


    ch = R_features[:,1]


    label =[]
    for i in range(len(ch)):
        if int(float(ch[i])) in set_array:
            label.append(1)
        else:
            label.append(0)
    return np.array(label)


In [49]:
def feature_extraction(file_name,threshold):
    
    data_names = pd.read_csv(file_name)
    file_names = data_names['Names']
    patient_id = data_names['MRD']
    mat_flie_path ='Patients/mat/'
    index_no = data_names['Channle no']
    patient_names = data_names['Names']
    #len(file_names)
    for i in range(len(file_names)):
        
        print(file_names[i]+'---->loading...')
        
        mat = mat73.loadmat(mat_flie_path+file_names[i])           
#         dict_y = dictionary(file_names[i])
        xx = mat['xx']
        try:
            fs = mat['fs'][0][0]
        except Exception:
            fs = mat['fs']
        t_hold = threshold
        #filtering
        xx_fastripples,xx_ripples = filter_(xx,fs)
        #features
        R_features,FR_features = R_FR_features(mat,xx_fastripples,xx_ripples,t_hold,fs,patient_id,i)
        #label
        
        R_label = label(R_features,index_no,i)
        FR_label = label(FR_features,index_no,i)
        R_features= np.hstack((R_features,R_label.reshape(len(R_label),1)))
        FR_features=np.hstack((FR_features,FR_label.reshape(len(FR_label),1)))
#         csv
        write_csv(file_names,i,R_features,FR_features)
        print(file_names[i]+'---->completed...')
    print('all---->completed')

In [50]:
feature_extraction('Patient list - Copy.csv',8)

ERROR:root:ERROR: MATLAB type not supported: string, (uint32)


Ezra.edf.mat---->loading...
Ezra.edf.mat---->completed...
all---->completed


In [33]:
def mobility(x):
    print(x)
    a,b = ent.hjorth_params 
    print(a)

x = [2,5,4,6,5,8,4,2,3,6,5]
print(ent.hjorth_params(x)[0])

1.348466206885108
